In [5]:
import pandas as pd
import os
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [66]:
## We'll be training a SVM model to predict the domain based on the utterance
## Take all the test datasets and combine to create the train data for the domain predition

raw=pd.concat([pd.read_csv(i) for i in os.listdir() if i.startswith('All_models_test')])
domain_map=raw[['utterance','domain']]
domain_map.rename(columns={'domain':'initial_domain'},inplace=True)

np.random.seed(500)
domain_map.dropna(subset=['utterance'],inplace=True)
domain_map['utterance']=domain_map['utterance'].str.lower()

Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(domain_map['utterance'],domain_map['initial_domain'],test_size=0.1)

Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.transform(Test_Y)

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(domain_map['utterance'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

SVM Accuracy Score ->  96.22641509433963


In [67]:
## Read the output file from the all domain test

df=pd.read_excel('All_outputs_combined.xlsx', engine='openpyxl')[['utterance','intent','intent1','intent_predicted','confidence','domain','Order of confidence scores']]

In [68]:
# take only the rows with top confidence scores

df=df[df['Order of confidence scores']==1]

In [69]:
df

utterance                                             intent  \
0         1bank delete  access_management.delete_application_account_o...   
3         1bank delete  access_management.delete_application_account_o...   
6         1bank delete  access_management.delete_application_account_o...   
9         1bank delete  access_management.delete_application_account_o...   
12        1bank delete  access_management.delete_application_account_o...   
...                ...                                                ...   
166830  zoomed font ie                            computer.zoomed_browser   
166833  zoomed font ie                            computer.zoomed_browser   
166836  zoomed font ie                            computer.zoomed_browser   
166839  zoomed font ie                            computer.zoomed_browser   
166842  zoomed font ie                            computer.zoomed_browser   

        intent1                                   intent_predicted  \
0            22                  datafirst.ddom_all_qry_starterkit   
3            22      cicd.easre_all_cicd_qry_dependancy_issues_1_0   
6            22                onboarding.can_i_change_my_1bank_id   
9            22       ablinrmdash.ibg4_in_qry_appstage_colladdress   
12           22  access_management.delete_application_account_o...   
...         ...                                                ...   
166830      219      training.dbs_learning_hub_mobile_app_features   
166833      219           ablindocs.ibg4_in_qry_preapp_rmchecklist   
166836      219                            computer.zoomed_browser   
166839      219        fcssaml.lcs_sg_advisory_fcis_record_details   
166842      219             udmapns.orm_udma_all_qry_lodgeudma_1_0   

        confidence   domain  Order of confidence scores  
0         0.528312     data                           1  
3         0.221229    easre                           1  
6         0.984716     hiri                           1  
9         0.089805  ibgpdts                           1  
12        0.890510     iris                           1  
...            ...      ...                         ...  
166830    0.097686     hiri                           1  
166833    0.390655  ibgpdts                           1  
166836    0.976525     iris                           1  
166839    0.110731      lcs                           1  
166842    0.228136  riskgov                           1  

[55615 rows x 7 columns]

In [70]:
# Make the prediction for the domain from the trained model above

df['predicted_domain']=Encoder.inverse_transform(SVM.predict(Tfidf_vect.transform(df['utterance'])))

In [71]:
df1=df[df['domain']==df['predicted_domain']].drop_duplicates('utterance')  ## take the only rows where predicted domain matches the model domains

In [72]:
## Calculate the accuracy score

df1['match']=df1['intent']==df1['intent_predicted']

k=df1['match'].value_counts()

print('The confidence score is : {}'.format(k[True]/(k[True]+k[False])))

The confidence score is : 0.9007372301211164


In [73]:
df1.to_excel('output.xlsx',index=None)